In [180]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [181]:
trainset = pd.read_csv('train.csv')
x_train = trainset[["Sex","Age","SibSp","Pclass","Parch"]].values
y_train = trainset.iloc[:, 1].values

In [182]:
testset=pd.read_csv('test.csv')
x_test = testset[["Sex","Age","SibSp","Pclass","Parch"]].values
y_test = testset.iloc[:, 1].values

In [183]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(x_train[:, 1:2])
x_train[:, 1:2] = imputer.transform(x_train[:, 1:2])
imputer.fit(x_test[:, 1:2])
x_test[:, 1:2] = imputer.transform(x_test[:, 1:2])


In [184]:

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x_train[:,0] = le.fit_transform(x_train[:,0])
x_test[:,0] = le.fit_transform(x_test[:,0])
print(x_train[:,:])

[[1 22.0 1 3 0]
 [0 38.0 1 1 0]
 [0 26.0 0 3 0]
 ...
 [0 29.69911764705882 1 3 2]
 [1 26.0 0 1 0]
 [1 32.0 0 3 0]]


In [185]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)


In [186]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0,C=1,gamma=0.1)
classifier.fit(x_train, y_train)

SVC(C=1, gamma=0.1, random_state=0)

In [187]:
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear']},
              {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search.fit(x_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

Best Accuracy: 82.72 %
Best Parameters: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}


In [188]:
y_pred = classifier.predict(x_test)


In [189]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[  0   0   0   0]
 [ 56  51   0   0]
 [ 61  32   0   0]
 [146  72   0   0]]


0.12200956937799043

In [190]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = x_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 82.72 %
Standard Deviation: 3.05 %


In [191]:
data={'PassengerId':testset["PassengerId"].values,
      'Survived':y_pred
}
df = pd.DataFrame(data, columns= ['PassengerId', 'Survived'])
df.to_csv (r'C:\Users\osheh\Desktop\last.csv', index = False, header=True)